In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

In [3]:
from sklearn import preprocessing

In [4]:
import re

In [4]:
file = pd.read_csv('preprocessed_data.csv')

In [5]:
file = pd.read_csv('test2.csv')

In [7]:
file = file.drop(['Unnamed: 0', 'Unnamed: 0.1', 'genres', 'keywords', 'original_language', 'overview',
                  'production_companies', 'production_countries', 'release_date', 'spoken_languages', 'tagline', 'title'],axis = 1)

## Normalize some columns

In [9]:
x = file.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
file = pd.DataFrame(x_scaled)

In [24]:
file.head()

,0,1,2,3,4,5,6,7,8,9,...,15095,15096,15097,15098,15099,15100,15101,15102,15103,15104
0,0.623684,0.171815,1.000000,0.479290,0.0,0.72,0.858057,0.404040,1.000000,0.300000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.789474,0.158846,0.344696,0.500000,0.0,0.69,0.327225,0.383838,0.363636,0.600000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.644737,0.122635,0.315884,0.437870,0.0,0.63,0.324753,0.464646,0.818182,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.657895,0.128272,0.389151,0.488166,0.0,0.76,0.662158,0.434343,0.545455,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.684211,0.050169,0.101916,0.390533,0.0,0.61,0.154450,0.434343,0.181818,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## K means

In [14]:
file = file.drop(['id'], axis=1)

In [11]:
data_list = file.values.tolist()

In [73]:
def point_avg(data, length):
    '''
    @params data The file that containes all the data that needs to be clustered.
    Their dimensions are expected to be larger than 2.
    @return new_center The center of all the points
    '''
    dimensions = length
    
    new_center = []
    
    for d in range(dimensions):
        dim_sum = 0 # sum of dimensions
        for p in data:
            dim_sum += p[d]
            
        new_center.append(dim_sum / float(len(data)))
        
    return new_center

In [79]:
def update_centers(data, assignments):
#     new_means = defaultdict(list)
    centers = []
#     for assignment, point in zip(assignments, data):
#         new_means[assignment].append(point)
        
    for points in data:
        centers.append(point_avg(points, len(points)))
        
    return centers

In [3]:
def assign_points(data_points, centers):
    assignments = []
    for point in data_points:
        shortest = 20000
        shortest_index = 0
        for i in range(len(centers)):
            val = distance(point, centers[i])
            if val < shortest:
                shortest = val
                shortest_index = i
        assignments.append(shortest_index)
    return assignments

In [4]:
def distance(a,b):
    dimensions = len(a)
    _sum = 0
    for d in range(dimensions):
        difference_sq = pow(a[d] - b[d], 2)
        _sum += difference_sq
    return sqrt(_sum)

In [5]:
def generate_k_points(data, k):
    centers = []
    
    dimensions = len(data[0])
    min_max = defaultdict(int)
    
    for point in data:
        for i in range(dimensions):
            val = point[i]
            min_key = 'min_' + str(i)
            max_key = 'max_' + str(i)
            if min_key not in min_max or val < min_max[min_key]:
                min_max[min_key] = val
            if max_key not in min_max or val > min_max[max_key]:
                min_max[max_key] = val
                
    for j in range(k):
        rand_point = []
        for i in range(dimensions):
            min_val = min_max['min_' + str(i)]
            max_val = min_max['max_' + str(i)]
            
            rand_point.append(uniform(min_val, max_val))
        centers.append(rand_point)
        
    return centers

In [6]:
def k_means(data, k):
    k_points = generate_k_points(data, k)
    assignments = assign_points(data, k_points)
    old_assignments = None
    while assignments != old_assignments:
        new_centers = update_centers(data, assignments)
        old_assignments = assignments
        assignments = assign_point(data, new_centers)
    return zip(assignments, data)

In [80]:
from collections import defaultdict
from random import uniform
from math import sqrt

points = [[1,2], [2,3], [3, 4], [4,5]]
print(k_means(points, 3))

TypeError: 'int' object is not subscriptable

In [27]:
class KMeansBase(object):

    def __init__(self, n_clusters = 8, init = "random", max_iter = 300, random_state = None, n_init = 10, tol = 1e-4):
        self.k = n_clusters # 聚类个数
        self.init = init # 输出化方式
        self.max_iter = max_iter # 最大迭代次数
        self.random_state = check_random_state(random_state) #随机数
        self.n_init = n_init # 进行多次聚类，选择最好的一次
        self.tol = tol # 停止聚类的阈值

    # fit对train建立模型
    def fit(self, dataset):
        self.tol = self._tolerance(dataset, self.tol)

        bestError = None
        bestCenters = None
        bestLabels = None
        for i in range(self.n_init):
            labels, centers, error = self._kmeans(dataset)
            if bestError == None or error < bestError:
                bestError = error
                bestCenters = centers
                bestLabels = labels
        self.centers = bestCenters
        return bestLabels, bestCenters, bestError

    # predict根据训练好的模型预测新的数据
    def predict(self, X):
        return self.update_labels_error(X, self.centers)[0]

    # 合并fit和predict
    def fit_predict(self, dataset):
        self.fit(dataset)
        return self.predict(dataset)

    # kmeans的主要方法，完成一次聚类的过程
    def _kmeans(self, dataset):
        self.dataset = np.array(dataset)
        bestError = None
        bestCenters = None
        bestLabels = None
        centerShiftTotal = 0
        centers = self._init_centroids(dataset)

        for i in range(self.max_iter):
            oldCenters = centers.copy()
            labels, error = self.update_labels_error(dataset, centers)
            centers = self.update_centers(dataset, labels)

            if bestError == None or error < bestError:
                bestLabels = labels.copy()
                bestCenters = centers.copy()
                bestError = error

            ## oldCenters和centers的偏移量
            centerShiftTotal = np.linalg.norm(oldCenters - centers) ** 2
            if centerShiftTotal <= self.tol:
                break

        #由于上面的循环，最后一步更新了centers，所以如果和旧的centers不一样的话，再更新一次labels，error
        if centerShiftTotal > 0:
            bestLabels, bestError = self.update_labels_error(dataset, bestCenters)

        return bestLabels, bestCenters, bestError


    # k个数据点，随机生成
    def _init_centroids(self, dataset):
        n_samples = dataset.shape[0]
        centers = []
        if self.init == "random":
            seeds = self.random_state.permutation(n_samples)[:self.k]
            centers = dataset[seeds]
        elif self.init == "k-means++":
            pass
        return np.array(centers)


    # 把tol和dataset相关联
    def _tolerance(self, dataset, tol):
        variances = np.var(dataset, axis=0)
        return np.mean(variances) * tol


    # 更新每个点的标签，和计算误差
    def update_labels_error(self, dataset, centers):
        labels = self.assign_points(dataset, centers)
        new_means = defaultdict(list)
        error = 0
        for assignment, point in zip(labels, dataset):
            new_means[assignment].append(point)

        for points in new_means.values():
            newCenter = np.mean(points, axis=0)
            error += np.sqrt(np.sum(np.square(points - newCenter)))

        return labels, error

    # 更新中心点
    def update_centers(self, dataset, labels):
        new_means = defaultdict(list)
        centers = []
        for assignment, point in zip(labels, dataset):
            new_means[assignment].append(point)

        for points in new_means.values():
            newCenter = np.mean(points, axis=0)
            centers.append(newCenter)

        return np.array(centers)


    # 分配每个点到最近的center
    def assign_points(self, dataset, centers):
        labels = []
        for point in dataset:
            shortest = float("inf")  # 正无穷
            shortest_index = 0
            for i in range(len(centers)):
                val = distance(point[np.newaxis], centers[i])
                if val < shortest:
                    shortest = val
                    shortest_index = i
            labels.append(shortest_index)
        return labels

In [33]:
# import KMeansBase
# from kmeans_plus import KMeansPlusPlus
import time

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn import datasets

import numpy as np

# from utils.misc_utils import distance, sortLabel
# from kmeans.kmeans_base import KMeansBase

import matplotlib.pyplot as plt

In [35]:
allData = {"iris":datasets.load_iris().data, "boston":datasets.load_boston().data, "digits":datasets.load_digits().data}

for dataName, value in allData.items():
    n_clusters = 3
    init = "random"
    if dataName == "digits":
        n_clusters = 10
    #降维
    tsne = TSNE(n_components=2, init='pca', random_state=0)
    X_tsne = tsne.fit_transform(value)
    labels = KMeansBase(n_clusters = n_clusters, init = init).fit_predict(value)
    takeTime = time.time() - startTime
    print("dataset=%s model=%s takeTime=%s"%(dataName, modelName, round(takeTime, 5)))
    plt.scatter(X_tsne[:, 0],X_tsne[:, 1], c=labels)
    plt.show()
    print("-" * 100)

NameError: name 'check_random_state' is not defined

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random

def init(data, k):
    data = list(data)
    return random.sample(data, k)

In [9]:
def min_dis(data, centroid_list):
    cluster_dict = dict()
    
    for item in data:
        p1 = item
        flag = -1
        min_dis = float("inf")
        
        for i in range(len(centroid_list)):
            p2 = centroid_list[i]
            dis = calc_dis(p1, p2)
            if dis < min_dis:
                flag = i
                
        if flag not in cluster_dict.keys():
            cluster_dict.setdefault(flag, [])
        cluster_dict[flag].append(item)
        
    return cluster_dict

In [10]:
def calc_dis(data, centroid_list):
    clusterDict = dict()  
    k = len(centroid_list)
    for item in data:
        p1 = item
        flag = -1
        minDis = float("inf")  
        for i in range(k):
            p2 = centroid_list[i]
            dis = calc_dis(p1, p2)  
            if dis < minDis:
                minDis = dis
                flag = i  
        if flag not in clusterDict.keys():
            clusterDict.setdefault(flag, [])
        clusterDict[flag].append(item)  
    return clusterDict  

In [11]:
def get_centroids(cluster_dict):
    import numpy as np
    centroid_list = []
    for key in cluster_dict.keys():
        centroid = np.mean(cluster_dict[key])
        centroid_list.append(centroid)
    # 得到新的质心的集合
    return centroid_list

In [12]:
def get_var(centroid_list, cluster_dict):
    # 将族类中各个向量与质心的距离累加求和
    sum = 0
    for k in centroid_list.keys():
        p1 = centroid_list[k]
        dis = 0
        for item in cluster_dict[k]:
            p2 = item
            dis += calc_dis(p1, p2)
        sum += dis
    return sum

In [13]:
def test():
    data = file

    centroid_list = init(data, 50)
    cluster_dict = min_dis(data, centroid_list)

    new_var = get_var(centroid_list, cluster_dict)
    old_var = 1 

    time = 2

    while abs(new_var - old_var) >= 0.00001:
        centroid_list = get_centroids(cluster_dict)
        cluster_dict = min(data, centroid_list)
        old_var = new_var
        new_var = get_var(centroid_list, cluster_dict)
        time += 1

        show_cluster(centroid_list, cluster_dict)

In [14]:
if __name__ == '__main__':
    test()

RecursionError: maximum recursion depth exceeded in comparison

In [1]:
import sys
sys.setrecursionlimit(2147483647)

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
type(iris.data)

numpy.ndarray

In [3]:
import numpy as np

np.ndarray(iris.data)

ValueError: sequence too large; cannot be greater than 32

In [5]:
import pandas as pd

file = pd.read_csv('H:\Downloads\movies.csv')

In [24]:
file = pd.read_csv('H:\Downloads\movies.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 